<a href="https://colab.research.google.com/github/zhangling297/Substance-Use/blob/master/Classification_fisher_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Funcgion f.lr.p() computes the probability vector under a logistic regression model



In [ ]:
f.lr.p <- function(X, beta) {
  # compute vector p of probabilities for logistic regression with logit link

  X <- as.matrix(X)
  beta <- as.vector(beta)
  p <- exp(X %*% beta) / (1 + exp(X %*% beta))
  return(p)
}

In [ ]:
1.3 Implementation # the counts y, the sample sizes m, and the probabilities p.

In [ ]:
f.lr.1 <- function(y, m, p) {
  #binomial log likelihood function
  #input: vectors: y = counts; m = sample sizes; p = probabilities
  # output: log-likelihood llike, a scalar

  llike <- t(y) %*% log(p) + t(m-y) %*% log(1-p)
  return(llike)
}

In [ ]:
f.lr.FS <- function(X, y, m, beta.1
                    , eps1 = 1e-6, eps2=1e-7, maxit = 50) {
  # Fisher's scoring routine for estimation of LR model with line search:
  # X =n-by-(r+1) design matrix
  # y =n-by-1 vector of success counts
  # m =n-by-1 vector of sample sizes
  # beta.1 = (r+1)-by-1 vector of starting values for regression est
  # Iteration controlled by:
  #  eps1 = absolute convergence criterion for beta
  #  eps2 = absolute convergence criterion for log-likelihood
  #  maxit = maximu. allowale number of interations
  # Output:
  #  out = list containing:
  #  beta.MLE = beta MLE
  #  NR.hist = iteration history of convergence differences
  #  beta.hist = iteration history of beta
  #  beta.cov = beta covariance matrix (inverse Fisher's information matrix at MLE)
  #  note = convergence note

  beta.2 <- rep(-Inf, length(beta.1)) # init beta.2
  diff.beta <- sqrt(sum((beta.1 - beta.2)^2)) # Euclidean distance

  llike.1 <- f.lr.1 (y, m, f.lr.p(X, beta.1)) # update loglikelihood
  llike.2 <- f.lr.1(y, m, f.lr.p(X, beta.2)) #update loglikelihood
  diff.like <- abs(llike.1 - llike.2) # diff
  if (is.nan(diff.like)) {diff.like <- 1e9 }
  i <- 1 # initial interation index
  alpha.step <- seq(-1, 2, by = 0.1)[-11]  # line search step sizes, excluding 0

  NR.hist <- data.frame (i, diff.beta, diff.like, llike.1, step.size = 1) # iteration history
  beta.hist <- matrix(beta.1, nrow = 1)
  while ((i <= maxit) & (diff.beta > eps1) & (diff.like > eps2)) {
    i <- i + 1  # increment iteration
    # update beta
    beta.2 <- beta.1 # old guess is current guess
    mu.2 <- m * f.lr.p(X, beta.2) # m*p is mean
    # variance matrix
    v.2 <- diag(as.vector(m * f.lr.p(X, beta.2) * (1 - f.lr.p(X, beta.2))))
    score.2 <- t(X) %*% (y -mu.2) # score function
    # this increment version inverts the informaiton matrix
      #Iinv.2 <- solve(t(X) %*% v.2 %*% X) # Inverse information matrix
      # increm <- Iinv.2 %*% score.2   # increment, solve() is inverse
      # this increment version solves for (beta.2-beta.1) whithout inverting Information
      increm <- solve(t(X) %*% v.2 %*% X, score.2) # solve for increment

      # Line search for improved step size
      llike.alpha.step <- rep(NA, length(alpha.step)) # init llike for line search
      for (i.alpha.step in 1:length(alpha.step)) {
        llike.alpha.step[i.alpha.step] <- f.lr.1(y, m
                          , f.lr.p(X, beta.2 + alpha.step[i.alpha.step] * increm))

      }
      # step size index for max incteade in log-likelihood (if tie, [1] takes first)
      ind.max.alpha.step <- which(llike.alpha.step == max(llike.alpha.step)) [1]

      beta.1 <- beta.2 + alpha.step[ind.max.alpha.step] * increm # update beta

      diff.beta <- sqrt(sum((beta.1 - beta.2)^2)) # Euclidean distance

      llike.2 <- llike.1
      llike.1 <- f.lr.1(y, m, f.lr.p(X, beta.1)) # update logliklihood
      diff.like <- abs(llike.1 - llike.2) # diff

      # interation history
      NR.hist <- rbind(NR.hist, c(i, diff.beta, diff.like, llike.1, alpha.step[ind.max.alpha.step]))
  }

  # prepare output

  out <-list()
  out$beta.MLE <- beta.1
  out$iter <- i-1
  out$NR.hist <- NR.hist
  out$beta.hist <- beta.hist
    v.1 <- diag(as.vector(m * f.lr.p(X, beta.1) * (1 - f.lr.p(X, beta.1))))
    Iinv.1 <- solve(t(X) %*% v.1 %*% X) # Inverse information matrix
  out$beta.cov <- Iinv.1

  if (!(diff.beta > eps1) & !(diff.like > eps2)) {
    out$note <- paste ("Absolute convergence of", eps1, "for betas and"
                    , eps2, "for log-likelihood satisfied")
                    }
    if (i > maxit) {
      out$note <- paste ("Exceeded max iterations of ", maxit)
    }
    return(out)
    }


In [ ]:
# Load the beetles dataset and fit qauadratic model

# conc = CS2 concertration
 # y = number of beetles killed
 # n = number of beetles exposed
 # rep = Replicate number (1 or 2)

 beet <- read.table("http://statacumen.com/teach/SC1/SC1_11_beetles.dat", header = TRUE)
 beet$rep <- factor (beet$rep)

 #create data variables: m, y, X
 n <- nrow(beet)
 m <- beet$n
 y <- beet$y
X.temp <- beet$conc

 # quadratic model
 X <- matrix (c(rep(1, n), X.temp, X.temp^2), nrow = n)
 colnames (X) <- c("Int", "conc", "conc2")
 r <- ncol(X) - 1 # number of regression coefficients -1
 # initial beta vector
 beta.1 <- c(log(sum(y) /sum(m-y)), rep(0, r))

 # fit betas using our Fisher Scoring function
 out <- f.lr.FS(X, y, m, beta.1)
 out
 # Print X and out$beta.MLE to verify
 print("X structure:")
 print(str(X))
 print("out$beta.MLE structure:")
 print(str(out$beta.MLE))

The routine converged in 6 iterations, at each step, the log-likelihood increased, and the norm of the difference between successive estimates eventually decreased to zero. The estimates are 0 for the econstant term for the linear term, and 0 for the quadratic term.



In [ ]:
# Create a parameter estimate table

beta.Est <- out$beta.MLE
beta.SE <- sqrt(diag(out$beta.cov)) # sqrt diag inverse Information matrix
beta.z <- beta.Est / beta.SE
beta.pval <- 2 * pnorm(-abs(beta.z))
beta.coef <- data.frame(beta.Est, beta.SE, beta.z, beta.pval)
beta.coef

In [ ]:
## compare to the glm() fit:
beet$conc2 <- beet$conc^2 # Add conc2 to the beet dataframe
glm.beetles2 <- glm(formula = cbind(y, m - y) ~ conc + conc2, family = binomial, data = beet)
summary(glm.beetles2)$call
summary(glm.beetles2)$coefficients
# Print glm.beetles2 to verify
print("glm.beetles2 structure:")
print(str(glm.beetles2))

In [ ]:
library(ggplot2)

# Calculate predicted probabilities for the FS model
p.hat.fs <- f.lr.p(X, out$beta.MLE)
print(paste("Length of p.hat.fs:", length(p.hat.fs)))

# Calculate predicted probabilities for the GLM model
p.hat.glm <- predict(glm.beetles2, type = "response")
print(paste("Length of p.hat.glm:", length(p.hat.glm)))

# Create a data frame for plotting, explicitly converting to vector
plot_data <- data.frame(
  conc = beet$conc,
  observed_mortality = beet$y / beet$n, # Corrected: use beet$n instead of beet$m
  predicted_fs = as.vector(p.hat.fs),
  predicted_glm = as.vector(p.hat.glm)
)

# Plot for Fisher Scoring (FS) Model
ggplot(plot_data, aes(x = conc)) +
  geom_point(aes(y = observed_mortality, color = "Observed"), size = 2) +
  geom_line(aes(y = predicted_fs, color = "Predicted (FS)"), linewidth = 1) +
  labs(
    title = "Observed vs. Predicted Mortality (Fisher Scoring Model)",
    x = "Concentration",
    y = "Mortality Probability",
    color = "Type"
  ) +
  scale_color_manual(values = c("Observed" = "blue", "Predicted (FS)" = "red")) +
  theme_minimal() +
  theme(legend.position = "bottom")

# Plot for GLM Model
ggplot(plot_data, aes(x = conc)) +
  geom_point(aes(y = observed_mortality, color = "Observed"), size = 2) +
  geom_line(aes(y = predicted_glm, color = "Predicted (GLM)"), linewidth = 1) +
  labs(
    title = "Observed vs. Predicted Mortality (GLM Model)",
    x = "Concentration",
    y = "Mortality Probability",
    color = "Type"
  ) +
  scale_color_manual(values = c("Observed" = "blue", "Predicted (GLM)" = "darkgreen")) +
  theme_minimal() +
  theme(legend.position = "bottom")

In [ ]:
# Example 2

# WBC is white cell blood count, is a binary factor or indicator variable AG
# AG + (1)  or AG-(0)  were classified
# NTOTAL is the number of patients with the given combination of AG and WBC
# NRES is the number of NTOTAL that survived at least one year form the time of diagnosis

# Modelling the probability p of surviving at least one year as a function of WBC and AG, the hypothesis is thea tWBC should be transdformed to a log scale m giben the skewness in the WBC value

## Leukemia white blood cell types example
# ntotal = number of patients with IAG and WBC combination
# nres = number surviving at least one year
# ag = 1 for AG+, 0 for AG-
# wbc = white cell blood count
# lwbc = log white cell blood count
# p.hat = Emperical Probability
leuk <- read.table("http://statacumen.com/teach/SC1/SC1_11_leuk.dat", header = TRUE)
leuk$lwbc <- log(leuk$wbc)
leuk$p.hat <- leuk$nres / leuk$ntotal

In [ ]:
# Fit the model with Fisher's Scoring method
# create data variables: m, y, X
n <- nrow(leuk)
m <- leuk$ntotal
y <- leuk$nres
X <- matrix(c(rep(1,n), leuk$lwbc, leuk$ag), nrow = n)
colnames(X) <- c("Int", "lwbc", "ag")

In [ ]:
r <- ncol(X) - 1 # number of regression coefficients - 1
# initial beta vector
beta.1 <- c(log(sum(y) / sum(m - y)), rep(0, r))
# fit betas using our Fisher Scoring function
out <- f.lr.FS(X, y, m, beta.1)
out



```
## compare to the glm() fit:
summary(glm.i.l)$call
## glm(formula = cbind(nres, ntotal - nres) ~ ag + lwbc, family = binomial,
## data = leuk)
summary(glm.i.l)$coefficients
```



In [ ]:
# create a parameter estimate table
beta.Est <- out$beta.MLE
beta.SE <- sqrt(diag(out$beta.cov)) # sqrt diag inverse Information matrix
beta.z <- beta.Est / beta.SE
beta.pval <- 2 * pnorm(-abs(beta.z))
beta.coef <- data.frame(beta.Est, beta.SE, beta.z, beta.pval)
beta.coef

In [ ]:

## compare to the glm() fit:
summary(glm.i.l)$call
## glm(formula = cbind(nres, ntotal - nres) ~ ag + lwbc, family = binomial,
## data = leuk)
summary(glm.i.l)$coefficients



```
## compare to the glm() fit:
summary(glm.i.l)$call
## glm(formula = cbind(nres, ntotal - nres) ~ ag + lwbc, family = binomial,
## data = leuk)
summary(glm.i.l)$coefficients
```



In [ ]:

# plot observed and predicted proportions
# leuk$p.hat calculated earlier
leuk$p.MLE <- f.lr.p(X, out$beta.MLE) #$
library(ggplot2)
p <- ggplot(leuk, aes(x = lwbc, y = p.hat, colour = ag))
p <- p + geom_line(aes(y = p.MLE))
# fitted values
p <- p + geom_point(aes(y = p.MLE), size=2)
# observed values
p <- p + geom_point(size = 2, alpha = 0.5)
p <- p + labs(title = "FS Observed and predicted probability of 1+ year survival")
print(p)